# Final Project - Finetuning Assistant

In [1]:
!pip install transformers==4.57.1
!pip install trl==0.24.0
!pip install peft==0.16.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 102.5 MB/s eta 0:00:0000:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 110.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.53.3
    Uninstalling transformers-4.53.3:
      Successfully uninstalled transformers-4.53.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 8.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 43.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.0 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2

In [2]:
import transformers, trl, peft

2025-12-05 10:58:35.795411: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764932316.005298      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764932316.059678      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
# imports
import numpy as np 
import json
import os
import shutil
import subprocess
import sys
from typing import List
from datasets import Dataset


from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, pipeline
#import transformers, trl, peft
import torch
import random
torch.manual_seed(3407); random.seed(3407); np.random.seed(3407)


from trl import SFTTrainer, SFTConfig
from peft import LoraConfig, get_peft_model, PeftModel


/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

## Get all the data

In [4]:
def setup_repo(repo_url: str, repo_name: str, work_dir: str = "/kaggle/working"):
    os.chdir(work_dir)
    
    # Remove repo if it exists
    if os.path.exists(os.path.join(work_dir, repo_name)):
        shutil.rmtree(os.path.join(work_dir, repo_name))
    
    # Clone repo
    subprocess.run(["git", "clone", repo_url], check=True)
    
    # Move into repo/data
    os.chdir(os.path.join(repo_name, "data"))


setup_repo("https://github.com/lkra/dstc11-track5.git", "dstc11-track5")


Cloning into 'dstc11-track5'...


In [5]:
## List all files in the current directory iteratively:
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./output_schema.json
./knowledge_aug_domain_reviews.json
./knowledge_aug_reviews.json
./knowledge.json
./README.md
./train/labels.json
./train/logs.json
./train/logs_bkp.json
./train/bkp/labels.json
./train/bkp/logs.json
./test/labels.json
./test/logs.json
./val/labels.json
./val/logs.json


In [6]:
with open('train/logs.json', 'r') as f:
    train_ds=json.load(f)

with open('train/labels.json', 'r') as f:
    labels=json.load(f)

with open('knowledge.json', 'r') as f:
    knowledge_base=json.load(f)

In [7]:
def format_dialogue(dialogue: List[dict]) -> List[dict]: 
    """
    Args:
    dialogue (List[dict]): A list of dictionaries where each dictionary contains two keys:
        - 'speaker' (str): A string indicating the speaker of the turn ('U' for user, 'S' for system).
        - 'text' (str): The text spoken by the respective speaker.

    Returns:
        List[dict]: A new array with a specific role and content

    """
    # Your solution here
    messages=[]
    messages.append({"role": "system", "content": "You are an assistant."})
    for dialogue_element in dialogue:
        role = "user" if dialogue_element['speaker'] == 'U' else "system"
        messages.append({"role": role, "content": dialogue_element['text']})

    return messages

In [8]:
def reformat_dataset(dataset, labels_dataset): 
    reformatted_dataset = {
        "messages": []
    }
    for sample_index in range(len(dataset)): 
        # Your solution here
        try:
            sample_dialogue = format_dialogue(dataset[sample_index])
            sample_response = labels_dataset[sample_index]['response']
            sample_dialogue.append({"role": "system", "content": sample_response})
            
            reformatted_dataset["messages"].append(sample_dialogue)
        except:
            continue


        
    return reformatted_dataset

reformatted_dataset = reformat_dataset(train_ds, labels)
dataset = Dataset.from_dict(reformatted_dataset)
dataset

Dataset({
    features: ['messages'],
    num_rows: 16897
})

In [9]:
def process_dataset_split(split: str) -> Dataset: 
    """Loads, reformats, and processes a dataset split for model training or evaluation.

    This function loads a dataset split (e.g., 'val', 'test') and generates a dataset for it, similar to what we had for the train split.

    Args:
        split (str): The name of the dataset split to process

    Returns:
        dataset: A HuggingFace `Dataset` object that contains the preprocessed and reformatted data for the specified split.

    """
    with open(f'{split}/logs.json', 'r') as f:
        data=json.load(f)

    with open(f'{split}/labels.json', 'r') as f:
        labels=json.load(f)

    data_ds = reformat_dataset(data, labels)
    new_dataset = Dataset.from_dict(data_ds)
    
    return new_dataset
    

validation_ds = process_dataset_split("val")
test_ds = process_dataset_split("test")

validation_ds, test_ds

(Dataset({
     features: ['messages'],
     num_rows: 2129
 }),
 Dataset({
     features: ['messages'],
     num_rows: 2798
 }))

In [10]:
model_id = "Qwen/Qwen3-1.7B"
tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
base = AutoModelForCausalLM.from_pretrained(model_id, dtype="auto", device_map="auto")

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [11]:
peft_cfg = LoraConfig(r=16, 
           lora_alpha=32, 
           lora_dropout = 0.05, 
           bias = "none", 
           use_rslora = False, 
           target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"])

model = get_peft_model(base, peft_cfg)

In [12]:
def pick_bf16():
    if torch.cuda.is_available():
        major, _ = torch.cuda.get_device_capability()
        return major >= 8
    return False

In [13]:
NUM_TRAIN_EPOCHS = 1
LEARNING_RATE    = 7e-5
WARMUP_STEPS     = ((2113 * NUM_TRAIN_EPOCHS)//100) * 10

os.environ["WANDB_DISABLED"] = "true"

sft_args = SFTConfig(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    logging_steps=10,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    max_length=1024,
    optim="adamw_torch_fused",
    fp16=not pick_bf16(),
    bf16=pick_bf16(),
    packing=False,
    dataset_num_proc=2,
    report_to="none",
    seed=3407
)

In [14]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,      
    eval_dataset=validation_ds,
    args=sft_args,
    processing_class=tok
)

Tokenizing train dataset (num_proc=2):   0%|          | 0/16897 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/16897 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/2129 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/2129 [00:00<?, ? examples/s]

In [15]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,5.571600
20,5.315900
30,4.395900
40,3.516400
50,2.864900
60,2.502800
70,2.209300
80,2.040700
90,1.960700
100,1.871800


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=2113, training_loss=1.3732818416436587, metrics={'train_runtime': 6148.8185, 'train_samples_per_second': 2.748, 'train_steps_per_second': 0.344, 'total_flos': 4.201868035718554e+16, 'train_loss': 1.3732818416436587, 'entropy': 1.2068944639629788, 'num_tokens': 3938417.0, 'mean_token_accuracy': 0.6782568560706245, 'epoch': 1.0})

## Create agent 1: Assistant 1

In [16]:
trainer.save_model("outputs/adapter")  
tok.save_pretrained("outputs/adapter")

('outputs/adapter/tokenizer_config.json',
 'outputs/adapter/special_tokens_map.json',
 'outputs/adapter/chat_template.jinja',
 'outputs/adapter/vocab.json',
 'outputs/adapter/merges.txt',
 'outputs/adapter/added_tokens.json',
 'outputs/adapter/tokenizer.json')

In [22]:
print(dataset[0:3])

{'messages': [[{'content': 'You are an assistant.', 'role': 'system'}, {'content': 'Can you help me find a place to stay that is moderately priced and includes free wifi?', 'role': 'user'}, {'content': 'sure, i have 17 options for you', 'role': 'system'}, {'content': "Are any of them in the south? I'd like free parking too.", 'role': 'user'}, {'content': 'Yes, two are in the south and both have free parking and internet. I recommend the Bridge Guesthouse. Would you like me to book a reservation?', 'role': 'system'}, {'content': 'I have back issues. Does this place have comfortable beds?', 'role': 'user'}, {'content': 'The Bridge Guest House is known for having pretty uncomfortable beds according to most guests. Only one guest found it to be comfortable.', 'role': 'system'}], [{'content': 'You are an assistant.', 'role': 'system'}, {'content': 'Do you have information about the Warkworth House?', 'role': 'user'}, {'content': 'Yes I do! The Warkworth House is a 4 star guesthouse that is 

In [23]:
NUM_TRAIN_EPOCHS = 1
LEARNING_RATE    = 7e-5
WARMUP_STEPS     = ((2113 * NUM_TRAIN_EPOCHS)//100) * 10

os.environ["WANDB_DISABLED"] = "true"

sft_args = SFTConfig(
    output_dir="outputs",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=LEARNING_RATE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=NUM_TRAIN_EPOCHS,
    logging_steps=10,
    lr_scheduler_type="linear",
    weight_decay=0.01,
    max_length=1024,
    optim="adamw_torch_fused",
    fp16=not pick_bf16(),
    bf16=pick_bf16(),
    packing=False,
    dataset_num_proc=2,
    report_to="none",
    seed=3407
)

In [24]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,      
    eval_dataset=validation_ds,
    args=sft_args,
    processing_class=tok
)

Tokenizing train dataset (num_proc=2):   0%|          | 0/16897 [00:00<?, ? examples/s]

Truncating train dataset (num_proc=2):   0%|          | 0/16897 [00:00<?, ? examples/s]

Tokenizing eval dataset (num_proc=2):   0%|          | 0/2129 [00:00<?, ? examples/s]

Truncating eval dataset (num_proc=2):   0%|          | 0/2129 [00:00<?, ? examples/s]

In [26]:
trainer.train()

Step,Training Loss
10,1.210900
20,1.232100
30,1.159600
40,1.177500
50,1.185800
60,1.176000
70,1.180300
80,1.160600
90,1.149700
100,1.162500


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


TrainOutput(global_step=2113, training_loss=1.1545771166886756, metrics={'train_runtime': 6163.3328, 'train_samples_per_second': 2.742, 'train_steps_per_second': 0.343, 'total_flos': 4.219805159713382e+16, 'train_loss': 1.1545771166886756, 'entropy': 1.1386982864803739, 'num_tokens': 3955363.0, 'mean_token_accuracy': 0.6888072027100457, 'epoch': 1.0})

In [27]:
trainer.save_model("outputs/adapter2")  
tok.save_pretrained("outputs/adapter2")

('outputs/adapter2/tokenizer_config.json',
 'outputs/adapter2/special_tokens_map.json',
 'outputs/adapter2/chat_template.jinja',
 'outputs/adapter2/vocab.json',
 'outputs/adapter2/merges.txt',
 'outputs/adapter2/added_tokens.json',
 'outputs/adapter2/tokenizer.json')

In [28]:
!zip -r "finetuned2.1.zip" "outputs/adapter"
!zip -r "finetuned2.2.zip" "outputs/adapter2"

  adding: outputs/adapter/ (stored 0%)
  adding: outputs/adapter/adapter_model.safetensors (deflated 7%)
  adding: outputs/adapter/tokenizer_config.json (deflated 90%)
  adding: outputs/adapter/tokenizer.json (deflated 81%)
  adding: outputs/adapter/adapter_config.json (deflated 57%)
  adding: outputs/adapter/README.md (deflated 66%)
  adding: outputs/adapter/vocab.json (deflated 61%)
  adding: outputs/adapter/merges.txt (deflated 57%)
  adding: outputs/adapter/added_tokens.json (deflated 68%)
  adding: outputs/adapter/special_tokens_map.json (deflated 69%)
  adding: outputs/adapter/chat_template.jinja (deflated 76%)
  adding: outputs/adapter/training_args.bin (deflated 51%)
  adding: outputs/adapter2/ (stored 0%)
  adding: outputs/adapter2/adapter_model.safetensors (deflated 8%)
  adding: outputs/adapter2/tokenizer_config.json (deflated 90%)
  adding: outputs/adapter2/tokenizer.json (deflated 81%)
  adding: outputs/adapter2/adapter_config.json (deflated 57%)
  adding: outputs/adapter2